In [1]:
import spikeinterface as si
import matplotlib.pyplot as plt
%matplotlib inline
import spikeinterface.extractors as se
import spikeinterface.preprocessing as spre
import spikeinterface.sorters as ss
import spikeinterface.widgets as sw
import spikeinterface.qualitymetrics as sqm
import json
import numpy as np
import pandas as pd
import seaborn as sns

from probeinterface import Probe, ProbeGroup
from probeinterface.plotting import plot_probe, plot_probegroup
from probeinterface import generate_dummy_probe, generate_linear_probe
from probeinterface import write_probeinterface, read_probeinterface
from probeinterface import write_prb, read_prb

/home/ubuntu/.conda/envs/spike_sorting_jct/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
recording_raw = se.read_blackrock(file_path='/media/ubuntu/sda/data/mouse6/ns4/natural_image/mouse6_022223_natural_image_001.ns4')
recording_recorded = recording_raw.remove_channels(["98", '31', '32'])
recording_stimulated = recording_raw.channel_slice(['98'])

probe_30channel = read_probeinterface('/media/ubuntu/sda/data/probe.json')
recording_recorded = recording_recorded.set_probegroup(probe_30channel)

recording_cmr = recording_recorded

In [52]:
traces_raw = recording_cmr.get_traces(start_frame=1000000, end_frame=1010000)

In [53]:
channel_indices = {
        "1": [1, 3, 5, 6, 9, 11],
        "2": [13, 15, 17, 19, 21, 23],
        "3": [24, 25, 26, 27, 28, 29],
        "4": [12, 14, 16, 18, 20, 22],
        "5": [0, 2, 4, 6, 8, 10]
        }

In [64]:
from matplotlib.backends.backend_pdf import PdfPages

colors = ["#7a2db3", "#94d868",
"#1a6df6", "#909f00", "#c883ff", "#00dd94",
"#ff6fe3", "#017e47", "#ce0070", "#01dbc8",
"#e1352b", "#009ccc", "#fa8219", "#0159a0",
"#f7bb66", "#86a2ff", "#a73200", "#c3bdff",
"#915b00", "#951b78", "#ddc567", "#ff679d",
"#ffaf6f", "#dda8d4", "#ff9857", "#ff8dbf",
"#7d4b32", "#ff6d59", "#b57390", "#8a3935"]

with PdfPages("traces_plot.pdf") as pdf:
    for arrary_name, ch_indices in channel_indices.items():
        fig, axes = plt.subplots(nrows=6, ncols=1, 
                                 figsize = (10, 10),
                                 sharex=True,
                                 gridspec_kw={'hspace':0.01})
        
        fig.suptitle(f'Array {arrary_name}', fontsize = 14, y = 0.92)

        for i, ch_idx in enumerate(ch_indices):
            ax = axes[i]

            ax.plot(range(traces_raw.shape[0]), traces_raw[:, ch_idx],
                    color = colors[ch_idx],
                    linewidth = 0.8)
            
            for spine in ax.spines.values():
                spine.set_visible(False)

            ax.tick_params(axis='both', which = 'both',
                           bottom = False, top = False,
                           left = False, right = False,
                           labelbottom = False, labelleft = False)
            
        plt.tight_layout()
        pdf.savefig(fig)
        plt.close()

/tmp/ipykernel_2042003/208784980.py:36: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [57]:
recording_f = spre.bandpass_filter(recording_recorded, freq_min=300, freq_max=3000)
traces_f = recording_f.get_traces(start_frame=1004500, end_frame=1005300)

In [63]:
with PdfPages("traces_filtered_plot.pdf") as pdf:
    for arrary_name, ch_indices in channel_indices.items():
        fig, axes = plt.subplots(nrows=6, ncols=1, 
                                 figsize = (10, 10),
                                 sharex=True,
                                 gridspec_kw={'hspace':0.001})
        
        fig.suptitle(f'Array {arrary_name}', fontsize = 14, y = 0.92)

        for i, ch_idx in enumerate(ch_indices):
            ax = axes[i]

            ax.plot(range(traces_f.shape[0]), traces_f[:, ch_idx],
                    color = colors[ch_idx],
                    linewidth = 0.8)
            
            for spine in ax.spines.values():
                spine.set_visible(False)

            ax.tick_params(axis='both', which = 'both',
                           bottom = False, top = False,
                           left = False, right = False,
                           labelbottom = False, labelleft = False)
            
        plt.tight_layout()
        pdf.savefig(fig)
        plt.close()

/tmp/ipykernel_2042003/4056515098.py:25: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [105]:
a = pd.read_csv("/media/ubuntu/sda/data/mouse6/output/02_merge_new/clusteri_inf_13.tsv", sep = '\t', index_col=0)

In [101]:
value = a.iloc[:, -63:-2]

In [102]:
import numpy as np

def calc_waveform_features(row):
    arr = row.values.astype(float)
    peak_idx = np.argmax(arr)
    trough_idx = np.argmin(arr)
    peak_amp = arr[peak_idx]
    trough_amp = arr[trough_idx]
    pt_ratio = peak_amp / trough_amp if trough_amp != 0 else np.nan

    # 找左右最近的0点
    zero_left = None
    zero_right = None
    # 左侧
    for i in range(peak_idx-1, -1, -1):
        if arr[i] == 0 or (arr[i] * arr[i+1] < 0):
            zero_left = i
            break
    # 右侧
    for i in range(peak_idx+1, len(arr)):
        if arr[i] == 0 or (arr[i] * arr[i-1] < 0):
            zero_right = i
            break

    # repolarization_slope: trough到左侧zero
    if zero_left is not None and trough_idx != zero_left:
        repolarization_slope = (arr[trough_idx] - arr[zero_left]) / (trough_idx - zero_left)
    else:
        repolarization_slope = np.nan

    # recovery_slope: peak到右侧zero
    if zero_right is not None and peak_idx != zero_right:
        recovery_slope = (arr[zero_right] - arr[peak_idx]) / (zero_right - peak_idx)
    else:
        recovery_slope = np.nan

    return pd.Series({
        'peak': peak_idx,
        'peak_amplitude': peak_amp,
        'Trough': trough_idx,
        'Trough_amplitude': trough_amp,
        'PT_ration': pt_ratio,
        'zero': (zero_left, zero_right),
        'repolarzation_slope': repolarization_slope,
        'recovery_slope': recovery_slope
    })

features = value.apply(calc_waveform_features, axis=1)


ValueError: could not convert string to float: '[28, 26, 29, 27, 19, 24]'

In [97]:
features

,peak,peak_amplitude,Trough,Trough_amplitude,PT_ration,zero,repolarzation_slope,recovery_slope
021322_8,25,2.836456,20,-5.660492,-0.501097,"(21, 31)",1.831224,-0.524914
021322_10,25,2.188837,20,-4.471668,-0.489490,"(22, 31)",2.054979,-0.386204
021322_12,26,3.579343,20,-6.635634,-0.539412,"(22, 32)",2.963973,-0.663156
021322_15,25,2.916263,20,-5.638270,-0.517227,"(21, 31)",2.449187,-0.548336
021322_20,25,6.264891,20,-11.231695,-0.557787,"(22, 31)",5.253734,-1.129706
...,...,...,...,...,...,...,...,...
37,0,NaN,0,NaN,NaN,"(None, None)",NaN,NaN
41,0,NaN,0,NaN,NaN,"(None, None)",NaN,NaN
42,0,NaN,0,NaN,NaN,"(None, None)",NaN,NaN
44,0,NaN,0,NaN,NaN,"(None, None)",NaN,NaN


In [98]:
a = pd.concat((a, features), axis=1)

In [99]:
a

,cluster,num_spikes,firing_rate,presence_ratio,snr,isi_violations_ratio,isi_violations_count,rp_contamination,rp_violations,sliding_rp_violation,...,cluster_date,index,peak,peak_amplitude,Trough,Trough_amplitude,PT_ration,zero,repolarzation_slope,recovery_slope
021322_8,8,66588,16.646958,1.0,8.470196,0.114871,382,0.102202,215,0.075,...,021322_8,NaN,25,2.836456,20,-5.660492,-0.501097,"(21, 31)",1.831224,-0.524914
021322_10,10,26414,6.603483,1.0,9.026696,0.072620,38,0.040971,14,0.035,...,021322_10,NaN,25,2.188837,20,-4.471668,-0.489490,"(22, 31)",2.054979,-0.386204
021322_12,12,34169,8.542229,1.0,10.563666,0.148463,130,0.160144,86,0.125,...,021322_12,NaN,26,3.579343,20,-6.635634,-0.539412,"(22, 32)",2.963973,-0.663156
021322_15,15,59947,14.986713,1.0,10.898887,0.074205,200,0.066789,116,0.050,...,021322_15,NaN,25,2.916263,20,-5.638270,-0.517227,"(21, 31)",2.449187,-0.548336
021322_20,20,48359,12.089720,1.0,18.395656,0.002851,5,0.001712,2,0.005,...,021322_20,NaN,25,6.264891,20,-11.231695,-0.557787,"(22, 31)",5.253734,-1.129706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37,52,36911,14.534549,1.0,10.822269,0.238590,384,0.283411,261,0.145,...,122022_52,52.0,0,NaN,0,NaN,NaN,"(None, None)",NaN,NaN
41,57,37273,14.677095,1.0,6.657995,0.238243,391,0.153438,155,0.080,...,122022_57,57.0,0,NaN,0,NaN,NaN,"(None, None)",NaN,NaN
42,58,36581,14.404604,1.0,13.796337,0.118927,188,0.114757,114,0.090,...,122022_58,58.0,0,NaN,0,NaN,NaN,"(None, None)",NaN,NaN
44,60,10151,3.997188,1.0,6.780570,0.369682,45,0.237748,17,0.190,...,122022_60,60.0,0,NaN,0,NaN,NaN,"(None, None)",NaN,NaN


In [92]:
neuron_df = a[a['Neuron'] == 'Neuron_3']

In [93]:
neuron_df

,cluster,num_spikes,firing_rate,presence_ratio,snr,isi_violations_ratio,isi_violations_count,rp_contamination,rp_violations,sliding_rp_violation,...,cluster_date,index,peak,peak_amplitude,Trough,Trough_amplitude,PT_ration,zero,repolarzation_slope,recovery_slope
021322_8,8,66588,16.646958,1.0,8.470196,0.114871,382,0.102202,215,0.075,...,021322_8,NaN,25,2.836456,20,-5.660492,-0.501097,"(21, 31)",1.831224,-0.524914
022522_12,12,45527,17.510317,1.0,9.497165,0.166836,399,0.159482,234,0.105,...,022522_12,NaN,25,3.576575,20,-6.564623,-0.544826,"(21, 30)",2.388984,-0.734510
6,8,42961,16.523398,1.0,11.094223,0.095793,204,0.079973,109,0.025,...,031722_8,8.0,0,NaN,0,NaN,NaN,"(None, None)",NaN,NaN
7,9,63709,15.927210,1.0,10.227547,0.086396,263,0.081147,158,0.045,...,042422_9,9.0,0,NaN,0,NaN,NaN,"(None, None)",NaN,NaN
6,7,40327,16.263867,1.0,11.186205,0.063529,125,0.052453,67,0.030,...,052422_7,7.0,0,NaN,0,NaN,NaN,"(None, None)",NaN,NaN
6,8,39062,15.023788,1.0,9.853332,0.082359,145,0.074330,84,0.030,...,062422_8,8.0,0,NaN,0,NaN,NaN,"(None, None)",NaN,NaN
10,13,42844,16.478398,1.0,9.741353,0.076959,163,0.060619,83,0.020,...,072322_13,13.0,0,NaN,0,NaN,NaN,"(None, None)",NaN,NaN
8,18,42692,16.673454,1.0,10.487757,0.071179,152,0.063102,87,0.035,...,082322_18,18.0,0,NaN,0,NaN,NaN,"(None, None)",NaN,NaN
3,4,40983,15.762632,1.0,10.138000,0.083075,161,0.073106,91,0.035,...,092422_4,4.0,0,NaN,0,NaN,NaN,"(None, None)",NaN,NaN
5,6,37251,15.521185,1.0,11.646549,0.119917,208,0.119570,130,0.060,...,102122_6,6.0,0,NaN,0,NaN,NaN,"(None, None)",NaN,NaN


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

metrics = [
    'peak', 'peak_amplitude', 'Trough', 'Trough_amplitude',
    'PT_ration', 'recovery_slope', 'repolarzation_slope'
]

dates = a['date'].unique()
neurons = a['Neuron'].unique()

with PdfPages("waveform_metrics_boxplot.pdf") as pdf:
    for metric in metrics:
        fig, ax = plt.subplots(figsize=(10, 6))
        # 先画每个Neuron的折线
        for neuron in neurons:
            neuron_df = a[a['Neuron'] == neuron]
            # 按date分组，取均值
            mean_per_date = neuron_df.groupby('date')[metric].mean()
            # 只保留有数据的日期
            ax.plot(
                valid_dates,
                mean_per_date.values,
                marker='o',
                color='gray',
                alpha=0.3,
                linewidth=1
            )
        # 再画箱线图
        box_data = []
        for date in dates:
            box_data.append(a[a['date'] == date][metric].dropna().values)
        ax.boxplot(box_data, positions=range(len(dates)), widths=0.5, patch_artist=True,
                   boxprops=dict(facecolor='lightblue', color='blue', alpha=0.7),
                   medianprops=dict(color='red'))
        ax.set_xticks(range(len(dates)))
        ax.set_xticklabels(dates, rotation=45)
        ax.set_ylabel(metric)
        ax.set_xlabel('Date')
        ax.set_title(f'{metric} by Date')
        plt.tight_layout()
        pdf.savefig(fig)
        plt.close(fig)

In [79]:
a['Neuron'].value_counts()

Neuron
Neuron_4     24
Neuron_6     23
Neuron_22    23
Neuron_5     19
Neuron_3     12
Neuron_18    12
Neuron_24    12
Neuron_23    12
Neuron_21    12
Neuron_19    12
Neuron_16    12
Neuron_17    12
Neuron_15    12
Neuron_14    12
Neuron_13    12
Neuron_11    12
Neuron_7     12
Neuron_25    12
Name: count, dtype: int64